<a href="https://colab.research.google.com/github/aleeepassarelli/scientific-validation-hub/blob/main/notebooks/01_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧪 Lab 01: Experiment Tracking com MLflow (Serverless)

**Framework:** Scientific Validation Hub
**Ferramenta:** [MLflow](https://mlflow.org/)
**Objetivo:** Rastrear versões de prompts e métricas de performance sem precisar configurar servidores complexos.

---

### ⚡ Quick Start (Modo Assistido)
1.  **Execute:** Rode todas as células abaixo (▶) para simular uma bateria de testes de um Agente.
2.  **Analise:** No final, geraremos uma tabela (DataFrame) com o histórico.
3.  **Audite:** Abra o Assistente Gemini e cole:

> "Atue como Cientista de Dados Sênior.
> Analise a tabela de experimentos 'results_df' gerada abaixo.
> 1. Qual versão do prompt ('prompt_version') teve a melhor relação entre 'quality_score' e 'latency_ms'?
> 2. Existe correlação entre a 'temperature' e o 'quality_score'?
> 3. Gere um insight sobre qual configuração devemos levar para produção."

# Instalando MLflow e dependências de análise
!pip install mlflow pandas matplotlib -q
print("✅ MLflow instalado. Pronto para iniciar o laboratório.")

In [ ]:
import mlflow
import pandas as pd
import random
import time
import uuid

# 1. Configuração do MLflow Local (Sem Servidor)
# Isso cria uma pasta ./mlruns no Colab para salvar os dados
mlflow.set_tracking_uri("file:./mlruns")
experiment_name = "Validacao_Agente_Vendas"
mlflow.set_experiment(experiment_name)

print(f"📁 Tracking URI configurado: file:./mlruns")
print(f"🧪 Experimento ativo: {experiment_name}")

# 2. Simulador de Agente (Mock)
# Em um caso real, aqui você chamaria a API do Gemini/OpenAI
def test_agent_performance(prompt_ver, model_type, temp):
    # Simulação: Modelos mais caros e temperaturas médias dão melhores resultados
    base_score = 0.85 if "gpt-4" in model_type else 0.65

    # Penalidade por temperatura extrema (muito baixa = robótico, muito alta = alucinação)
    temp_penalty = abs(0.5 - temp) * 0.2

    # Ruído aleatório (variância natural da IA)
    noise = random.uniform(-0.05, 0.05)

    final_score = base_score - temp_penalty + noise

    # Simulação de Latência e Tokens
    latency = random.randint(800, 2500) if "gpt-4" in model_type else random.randint(200, 800)
    tokens = 150 + random.randint(-20, 50)

    return {
        "quality": min(1.0, max(0.0, final_score)), # Clamp 0-1
        "latency": latency,
        "tokens": tokens
    }

# 3. Bateria de Testes (Grid Search Simplificado)
configs = [
    {"ver": "v1_agressiva", "model": "gpt-3.5-turbo", "temp": 0.9},
    {"ver": "v1_agressiva", "model": "gpt-4",         "temp": 0.9},
    {"ver": "v2_consultiva", "model": "gpt-3.5-turbo", "temp": 0.2},
    {"ver": "v2_consultiva", "model": "gpt-4",         "temp": 0.5}, # Sweet spot teórico
    {"ver": "v3_minimalista", "model": "gpt-4",        "temp": 0.1},
]

print("\n🚀 Iniciando execuções...")

for i, conf in enumerate(configs):
    run_name = f"run_{i}_{conf['ver']}"

    with mlflow.start_run(run_name=run_name):
        # A. Log de Parâmetros (O que mudamos)
        mlflow.log_param("prompt_version", conf["ver"])
        mlflow.log_param("model", conf["model"])
        mlflow.log_param("temperature", conf["temp"])

        # B. Execução
        print(f"   --> Testando: {conf['ver']} no {conf['model']}...")
        metrics = test_agent_performance(conf["ver"], conf["model"], conf["temp"])

        # C. Log de Métricas (O resultado)
        mlflow.log_metric("quality_score", metrics["quality"])
        mlflow.log_metric("latency_ms", metrics["latency"])
        mlflow.log_metric("cost_tokens", metrics["tokens"])

        # D. Log de Artefatos (Opcional - ex: o texto gerado)
        with open("output.txt", "w") as f:
            f.write(f"Simulação de output para {conf['ver']}")
        mlflow.log_artifact("output.txt")

print("\n✅ Ciclo de experimentos finalizado.")